In [1]:
import pandas as pd

tx = pd.read_csv("../data/transactions_scored.csv", parse_dates=["timestamp"])
tx.head()


,tx_id,timestamp,from_wallet,to_wallet,amount,chain,tx_type,is_fraud_pattern,pattern_tags,from_wallet_age_days,from_country,from_is_exchange_linked,to_wallet_age_days,to_country,to_is_exchange_linked,risk_score,risk_bucket
0,TX_025754,2025-09-12 20:56:27.419784,WALLET_01731,WALLET_01773,33.59,ETH,transfer,1,;mixing_like_outbound,235,CA,1,1,IN,1,30,MEDIUM
1,TX_027846,2025-09-12 21:09:01.419784,WALLET_00342,WALLET_00331,20.49,USDC,transfer,0,NaN,517,US,0,546,SG,0,0,LOW
2,TX_011013,2025-09-12 21:13:44.419784,WALLET_01933,WALLET_00939,35.94,BTC,contract_interaction,1,;mixing_like_outbound;high_risk_counterparty,496,BR,0,5,DE,1,80,HIGH
3,TX_019879,2025-09-12 21:20:41.419784,WALLET_01541,WALLET_01895,13.36,ETH,transfer,0,NaN,331,CA,1,230,BR,0,0,LOW
4,TX_000422,2025-09-12 21:21:41.419784,WALLET_01106,WALLET_00645,76.97,ETH,swap,0,NaN,126,IN,1,507,GB,0,0,LOW


In [2]:
tx.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   tx_id                    30000 non-null  object        
 1   timestamp                30000 non-null  datetime64[ns]
 2   from_wallet              30000 non-null  object        
 3   to_wallet                30000 non-null  object        
 4   amount                   30000 non-null  float64       
 5   chain                    30000 non-null  object        
 6   tx_type                  30000 non-null  object        
 7   is_fraud_pattern         30000 non-null  int64         
 8   pattern_tags             2397 non-null   object        
 9   from_wallet_age_days     30000 non-null  int64         
 10  from_country             30000 non-null  object        
 11  from_is_exchange_linked  30000 non-null  int64         
 12  to_wallet_age_days       30000 n

In [3]:
tx.describe()


,timestamp,amount,is_fraud_pattern,from_wallet_age_days,from_is_exchange_linked,to_wallet_age_days,to_is_exchange_linked,risk_score
count,30000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,2025-10-28 02:45:27.622517504,12.204762,0.079900,348.995167,0.191167,350.771867,0.190300,5.618000
min,2025-09-12 20:56:27.419784,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-10-05 16:39:46.919783936,3.780000,0.000000,162.000000,0.000000,164.000000,0.000000,0.000000
50%,2025-10-28 03:22:42.419783936,7.340000,0.000000,343.000000,0.000000,347.000000,0.000000,0.000000
75%,2025-11-19 16:47:08.169783808,14.530000,0.000000,525.000000,0.000000,525.000000,0.000000,10.000000
max,2025-12-11 20:46:26.419784,576.270000,1.000000,719.000000,1.000000,719.000000,1.000000,80.000000
std,NaN,16.019191,0.271143,210.679524,0.393227,210.043853,0.392544,10.179794


In [4]:
fraud_pattern_rate = tx["is_fraud_pattern"].mean()
print(f"Fraud pattern rate: {fraud_pattern_rate:.2%}")


Fraud pattern rate: 7.99%


In [5]:
tx["risk_bucket"].value_counts(normalize=True).mul(100).round(1)


risk_bucket
LOW       92.0
MEDIUM     7.9
HIGH       0.1
Name: proportion, dtype: float64

In [6]:
chain_summary = (
    tx.groupby("chain")
    .agg(
        tx_count=("tx_id", "count"),
        average_risk=("risk_score", "mean"),
        fraud_pattern_rate=("is_fraud_pattern", "mean")
    )
    .reset_index()
)

chain_summary


,chain,tx_count,average_risk,fraud_pattern_rate
0,BTC,8981,12.683443,0.082062
1,ETH,11888,2.593371,0.079240
2,USDC,9131,2.606505,0.078633


In [7]:
top_risky_wallets = (
    tx.groupby("from_wallet")
    .agg(
        tx_count=("tx_id", "count"),
        average_risk=("risk_score", "mean"),
        max_risk=("risk_score", "max"),
        fraud_pattern_rate=("is_fraud_pattern", "mean")
    )
    .reset_index()
    .sort_values("max_risk", ascending=False)
    .head(20)
)

top_risky_wallets


,from_wallet,tx_count,average_risk,max_risk,fraud_pattern_rate
938,WALLET_00939,13,46.153846,80,1.000000
1932,WALLET_01933,16,8.750000,80,0.062500
181,WALLET_00182,10,48.000000,80,1.000000
1986,WALLET_01987,19,44.736842,80,1.000000
48,WALLET_00049,16,43.750000,80,1.000000
1818,WALLET_01819,17,44.705882,80,1.000000
1277,WALLET_01278,13,7.692308,80,0.076923
1975,WALLET_01976,17,45.882353,70,1.000000
1829,WALLET_01830,15,8.000000,70,0.066667
854,WALLET_00855,22,7.272727,70,0.045455


## Key Takeaways

- Fraud pattern rate gives a quick sense of how often synthetic patterns fire.
- Certain chains show higher risk due to how patterns were injected.
- High-risk wallets tend to be:
  - Very new
  - Interacting with pattern-heavy wallets
  - Transacting larger amounts
- This notebook supports interview explanations about:
  - Risk aggregation
  - Wallet-level suspicious behavior
  - Differences between crypto transaction risk and ACH/card risk
